## Image Classification

### Download dependencies

In [2]:
# pip install numpy

  Using cached numpy-2.2.4-cp313-cp313-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.4-cp313-cp313-win_amd64.whl (12.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# pip uninstall torch torchvision torchaudio -y

In [ ]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/rocm6.2.4

In [3]:
# pip install torch torchvision torchaudio

  Using cached torch-2.6.0-cp313-cp313-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp313-cp313-win_amd64.whl.metadata (6.3 kB)
  Using cached torchaudio-2.6.0-cp313-cp313-win_amd64.whl.metadata (6.7 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached pillow-11.1.0-cp313-cp313-win_amd64.whl.metadata (9.3 kB)
  Using cached MarkupSafe-3.0.2-cp313-cp313-win_amd64.whl.metadata (4.1 kB)
Using cached torch-2.6.0-cp313-cp313-win_amd64.whl (204.1 MB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached torchvision-0.21.0-cp313-cp313-win_amd64.

In [4]:
pip install tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
from PIL import Image
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

In [6]:
print(torch.cuda.is_available())

False


In [7]:
transform = transforms.Compose([
    transforms.Resize(256),            
    transforms.CenterCrop(224), 
    transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip images
    transforms.RandomRotation(20),  # Rotate images by up to 20 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1)), # Slight translations
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])


In [8]:
dataset = torchvision.datasets.ImageFolder(root='dataset/Images', transform=transform)

train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size

train_data, test_data = random_split(dataset, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True, num_workers=2)

In [9]:
image, label = train_data[0]

In [10]:
image.size()

torch.Size([3, 224, 224])

In [11]:
class_name = ['Action','Horror','Romance','Sci-Fi','Sports']

In [12]:
class NeuralNet(nn.Module):

    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5) # (12, 220, 220)
        self.pool = nn.MaxPool2d(2, 2) # (12, 110, 110)
        self.conv2 = nn.Conv2d(12, 24, 5) # (24, 106, 106) -> (24, 53, 53)

        self.fc1 = nn.Linear(24 * 53 * 53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  
        x = self.pool(F.relu(self.conv2(x)))  
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))         
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [13]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)

In [14]:
from tqdm import tqdm

num_of_epochs = 800

for epoch in range(num_of_epochs):
    
    running_loss = 0.0
    for i, data in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}")):
        
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)

    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}/{num_of_epochs} - Loss: {avg_loss:.4f}")


Epoch 1:   0%|          | 0/14 [00:00<?, ?it/s]/home/ray/Desktop/Anime_Recommender_SystemV2/.venv/lib/python3.12/site-packages/torch/nn/modules/linear.py:125: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:310.)
  return F.linear(input, self.weight, self.bias)
Epoch 20: 100%|██████████| 14/14 [00:01<00:00, 10.01it/s]


Epoch 20/800 - Loss: 1.2478


Epoch 40: 100%|██████████| 14/14 [00:01<00:00, 10.44it/s]


Epoch 40/800 - Loss: 1.1618


Epoch 60: 100%|██████████| 14/14 [00:01<00:00,  9.67it/s]


...

Epoch 760/800 - Loss: 0.0682


Epoch 780: 100%|██████████| 14/14 [00:01<00:00, 10.20it/s]


Epoch 780/800 - Loss: 0.0918


Epoch 800: 100%|██████████| 14/14 [00:01<00:00, 10.13it/s]

Epoch 800/800 - Loss: 0.0809


In [15]:
torch.save(net.state_dict(), 'models/trained_image_model_SGDV4.pth')

In [14]:
net = NeuralNet()
net.load_state_dict(torch.load('models/trained_image_model_SGDV4.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [17]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 97.87%


In [18]:
new_transform = transforms.Compose([
    transforms.Resize(256),            
    transforms.CenterCrop(224), 
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

image_paths = ['images/sports/100.jpeg']

images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f"Prediction: {class_name[predicted.item()]}")

Prediction: Sports
